#Melspectrograms extraction

In [ ]:
!pip install librosa==0.8.0

In [ ]:
import librosa

In [ ]:
import wave
import pandas as pd
import matplotlib.image as mpimg
def scale_minmax(X, min=0.0, max=1.0):
    X_std = (X - X.min()) / (X.max() - X.min())
    X_scaled = X_std * (max - min) + min
    return X_scaled

In [ ]:
path_to_folder = '/content/drive/My Drive/ComParE2020_Breathing/'
wav_files_path = '/content/drive/My Drive/ComParE2020_Breathing/wav/'
mel_path = '/content/drive/My Drive/ComParE2020_Breathing/melspectrograms/'


In [ ]:

def extract_mel(path,file,save_path):
    sr = 16000
    hop_length = 320 # step per window
    n_fft=640   # size of window
    n_mels=128  # num of mels
    if os.path.isfile(path+file):
        sound, sr = librosa.load(path + file, sr=sr)
    
        S = librosa.feature.melspectrogram(sound, sr=sr, n_fft=n_fft,
                                           hop_length=hop_length,
                                           n_mels=n_mels,
                                           center=True)
        S_DB = librosa.power_to_db(S, ref=np.max)
        img = scale_minmax(S_DB, 0, 255).astype(np.uint8)
        img = np.flip(img, axis=0)
        img = 255-img
        new_arr = np.hstack([img[:,1:50][:,::-1],img])
        new_arr = np.hstack([new_arr,img[:,-50:-1][:,::-1]])
        np.save(save_path+file.replace(".wav",".npy"), new_arr)
        print(save_path+file.replace(".wav",".npy")," saved")
        return img
        
def run_mel_extraction(path,save_path):
    if not os.path.exists(os.path.dirname(save_path)):
        dir_name = os.path.dirname(save_path)
        os.makedirs(dir_name)
    files = os.listdir(path)
    for file in files:
      extract_mel(path,file,save_path)
      


In [ ]:
run_mel_extraction(wav_files_path,mel_path)

#Main

In [ ]:
import sys
sys.path.append(path_to_folder)

In [ ]:
from ComPare2020 import *

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 8529868149378632122
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 448417981364719811
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 5142803346505871553
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15695549568
locality {
  bus_id: 1
  links {
  }
}
incarnation: 6995272799591259717
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"
]


In [ ]:
if "Tesla P100-PCIE-16GB" not in device_lib.list_local_devices()[-1].physical_device_desc:
  print("----To run the model you need Tesla P100!----")

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git
from classification_models.tfkeras import Classifiers

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-wz3w8qyg
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-wz3w8qyg
  Running command git submodule update --init --recursive -q
     |████████████████████████████████| 51kB 1.6MB/s 
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=20032 sha256=fac432b07b36cab6eb72a626b7dfac08a474e0fba67672132de0fe28b074e0da
  Stored in directory: /tmp/pip-ephem-wheel-cache-e6jqxdfz/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [ ]:
!pip install --upgrade tables

     |████████████████████████████████| 4.3MB 3.5MB/s 
  Found existing installation: tables 3.4.4
    Uninstalling tables-3.4.4:
      Successfully uninstalled tables-3.4.4


#4 submission

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
ensembel_models_paths = ["/content/drive/My Drive/ComParE2020_Breathing/models_4_submit/fold_0.h5",
                     "/content/drive/My Drive/ComParE2020_Breathing/models_4_submit/fold_1.h5",
                     "/content/drive/My Drive/ComParE2020_Breathing/models_4_submit/fold_2.h5",
                     "/content/drive/My Drive/ComParE2020_Breathing/models_4_submit/fold_3.h5"]

In [ ]:
X_test = get_X_melspactrograms("test",mel_path)
df_meta_test = get_meta_info("test",path_to_folder+"labels.csv")

(128, 193584)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2822: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [ ]:
reset_random_seeds()
ResNet18, preprocess_input = Classifiers.get('resnet18')
del ResNet18

In [ ]:
models_scores = dict()
for i in [0,1,2,3]:
    model_loaded = load_model(ensembel_models_paths[i],custom_objects={"correlation_coefficient_loss":correlation_coefficient_loss})
    models_scores[i] = get_predict_test(model_loaded,X_test,df_meta_test,1,preprocess_input)
    del model_loaded

In [ ]:
predictions = pd.concat(models_scores,axis=1)
predictions["mean"] = predictions.mean(axis=1)


In [ ]:
predictions["mean"].to_csv("test_predictions_4trial.csv")